##### Capstone Project1: Identifying counties that are the most fire-prone and predicting the cause of a fire wildfire. 

DataSet: 1.88 Million US Wildfires (Kaggle1)

This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed, and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24 years.


In this documnet I have mentioned and explained the data wrangling steps I took to clean the dataset

Step 1: As the dataset was in SQL format, I did some cleaning activity in SQLite with Query.
These were many columns in SQL table "fires" which was related to Source of data and agency informaton which collected or prepared report on Data. I omited these columns and imported only relevamt data to "US_Wildfire_data.csv" file.
Also while importing I filled in all the null/blank values with "Nan" value.


select FOD_ID,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID
,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR ,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,
FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME from Fires

In [2]:
#importing all required packages
import pandas as pd
import numpy as np

# Reading US_WildFire data from .csv file to DataFrame
US_fires = pd.read_csv('US_Wildfire_data.csv',low_memory=False)

In [3]:
US_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 31 columns):
Unnamed: 0                 int64
Unnamed: 0.1               int64
FOD_ID                     int64
LOCAL_FIRE_REPORT_ID       object
LOCAL_INCIDENT_ID          object
FIRE_CODE                  object
FIRE_NAME                  object
ICS_209_INCIDENT_NUMBER    object
ICS_209_NAME               object
MTBS_ID                    object
MTBS_FIRE_NAME             object
COMPLEX_NAME               object
FIRE_YEAR                  int64
DISCOVERY_DATE             float64
DISCOVERY_DOY              int64
DISCOVERY_TIME             float64
STAT_CAUSE_CODE            int64
STAT_CAUSE_DESCR           object
CONT_DATE                  float64
CONT_DOY                   float64
CONT_TIME                  float64
FIRE_SIZE                  float64
FIRE_SIZE_CLASS            object
LATITUDE                   float64
LONGITUDE                  float64
OWNER_CODE                 int64
OW

Step 2: By looking at the info of dataframe I observed many columns with more than 50% of its data with null values. As those columns were not helping data set I omitted those columns.

In [4]:
# Picking columns which have more than 50% of total data 
US_fires =  US_fires[['FOD_ID','OWNER_DESCR','DISCOVERY_DATE','DISCOVERY_DOY','STAT_CAUSE_DESCR','FIRE_YEAR','FIRE_SIZE','FIRE_SIZE_CLASS','LATITUDE','LONGITUDE','STATE','FIPS_NAME']]
US_fires = US_fires.rename(columns={'FIPS_NAME':'COUNTY_NAME'})

In [5]:
US_fires.head(10)

,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME
0,1,USFS,2453403.5,33,Miscellaneous,2005,0.10,A,40.036944,-121.005833,CA,Plumas
1,2,USFS,2453137.5,133,Lightning,2004,0.25,A,38.933056,-120.404444,CA,Placer
2,3,STATE OR PRIVATE,2453156.5,152,Debris Burning,2004,0.10,A,38.984167,-120.735556,CA,El Dorado
3,4,USFS,2453184.5,180,Lightning,2004,0.10,A,38.559167,-119.913333,CA,Alpine
4,5,USFS,2453184.5,180,Lightning,2004,0.10,A,38.559167,-119.933056,CA,Alpine
5,6,USFS,2453186.5,182,Lightning,2004,0.10,A,38.635278,-120.103611,CA,Amador
6,7,USFS,2453187.5,183,Lightning,2004,0.10,A,38.688333,-120.153333,CA,El Dorado
7,8,STATE OR PRIVATE,2453437.5,67,Debris Burning,2005,0.80,B,40.968056,-122.433889,CA,NaN
8,9,STATE OR PRIVATE,2453444.5,74,Debris Burning,2005,1.00,B,41.233611,-122.283333,CA,NaN
9,10,USFS,2453187.5,183,Lightning,2004,0.10,A,38.548333,-120.149167,CA,Amador


In [6]:
US_fires.shape

(1880465, 12)

Step3: As County_Name  is the column which I am interested in, I checked if there are any Nan values in that column.

In [7]:
US_fires['COUNTY_NAME'].value_counts(dropna=False)

NaN                 678148
Washington           11014
Lincoln              10571
Jackson               9902
Marion                8908
                     ...  
Buena Vista City         1
Red Lake                 1
Wells                    1
Fairfax                  1
Love                     1
Name: COUNTY_NAME, Length: 1698, dtype: int64


Step 4: Filling the missing values of COUNTY_NAME column using 'LATITUDE' and 'LONGITUDE' columns. 
To do this I assigned rows with "Nan" values in FIPS_NAME(County name) to new dataframe and merged values of 'LATITUDE' and 'LONGITUDE' columns into new column.
Also reIndexing DataFrame

In [8]:
# Assigned all the rows which have no county names to new DataFrame 
nullCountyNameDF = US_fires[US_fires['COUNTY_NAME'].isnull()]


# zipped Latitude and Longitude into new column
nullCountyNameDF['LatLong'] = list(zip(nullCountyNameDF.LATITUDE, nullCountyNameDF.LONGITUDE))

# reindexing the new DataFrame
nullCountyNameDF.index = pd.RangeIndex(len(nullCountyNameDF.index))

C:\Users\Lasya\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
nullCountyNameDF.head()


,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME,LatLong
0,8,STATE OR PRIVATE,2453437.5,67,Debris Burning,2005,0.8,B,40.968056,-122.433889,CA,NaN,"(40.96805556, -122.43388889)"
1,9,STATE OR PRIVATE,2453444.5,74,Debris Burning,2005,1.0,B,41.233611,-122.283333,CA,NaN,"(41.23361111, -122.28333333)"
2,48,USFS,2453535.5,165,Lightning,2005,0.5,B,39.292222,-105.183056,CO,NaN,"(39.29222222, -105.18305556)"
3,62,USFS,2453520.5,150,Lightning,2005,0.2,A,38.913333,-105.983611,CO,NaN,"(38.91333333, -105.98361111)"
4,63,USFS,2453542.5,172,Campfire,2005,0.1,A,39.100278,-106.367500,CO,NaN,"(39.10027778, -106.3675)"


In [10]:
nullCountyNameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678148 entries, 0 to 678147
Data columns (total 13 columns):
FOD_ID              678148 non-null int64
OWNER_DESCR         678148 non-null object
DISCOVERY_DATE      678148 non-null float64
DISCOVERY_DOY       678148 non-null int64
STAT_CAUSE_DESCR    678148 non-null object
FIRE_YEAR           678148 non-null int64
FIRE_SIZE           678148 non-null float64
FIRE_SIZE_CLASS     678148 non-null object
LATITUDE            678148 non-null float64
LONGITUDE           678148 non-null float64
STATE               678148 non-null object
COUNTY_NAME         0 non-null object
LatLong             678148 non-null object
dtypes: float64(4), int64(3), object(6)
memory usage: 67.3+ MB


Step 5: Getting CountyName by Passing Latitude and Longitude Value as tuple to "reverse_geocoder.search" method as parameters.
As county value had "County" as suffix, truncated it from column values.

In [11]:
# importing geocoder package
import reverse_geocoder as rg

# converted LatLong tuple column to a list and passed this a parameter for search method.
coordinates = nullCountyNameDF['LatLong'].to_list()
newCountyNames = rg.search(coordinates)

# Converted the output from list to a dataframe and renamed column names to appropriate names.
newCountyNamesDF = pd.DataFrame(newCountyNames)
newCountyNamesDF = newCountyNamesDF.rename(columns={'admin1':'State','admin2':'County'})

# striped suffix "County" from values of column "County"
newCountyNamesDF['County'] = newCountyNamesDF['County'].map(lambda x: x.replace(' County',''))
newCountyNamesDF

Loading formatted geocoded file...


,lat,lon,name,State,County,cc
0,41.20821,-122.27195,Dunsmuir,California,Siskiyou,US
1,41.20821,-122.27195,Dunsmuir,California,Siskiyou,US
2,39.25666,-104.99248,Perry Park,Colorado,Douglas,US
3,38.84222,-106.13113,Buena Vista,Colorado,Chaffee,US
4,39.25082,-106.29252,Leadville,Colorado,Lake,US
...,...,...,...,...,...,...
678143,40.44821,-122.29778,Anderson,California,Shasta,US
678144,37.59493,-120.95771,Ceres,California,Stanislaus,US
678145,37.59493,-120.95771,Ceres,California,Stanislaus,US
678146,37.63826,-120.90215,Empire,California,Stanislaus,US


Step 6: At first look I observed few of the county names are not matching with expected states in "noCountyNameDF" DataFrame.(Note: When latitude and longitude lies in US state border this discripency of value is observed).
To get rows with the mismatching State value, new column is mapped with State name and State Code as dictionary with former being key.

In [12]:
# Reading a file to get all USA states with its name and abrreviation as columns
zipcode = pd.read_csv('states.csv',low_memory=False)

# Build a dictionary with State names as key and its Abbreviation as value.
zipcodeDict = dict(zip(zipcode['State'], zipcode['Abbreviation']))
print(zipcodeDict)

{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [13]:
# map State names from dataFrame newCountyNamesDF with dictionary zipcodeDict
newCountyNamesDF['state_code'] = newCountyNamesDF['State'].map(zipcodeDict)
newCountyNamesDF.head(5)

,lat,lon,name,State,County,cc,state_code
0,41.20821,-122.27195,Dunsmuir,California,Siskiyou,US,CA
1,41.20821,-122.27195,Dunsmuir,California,Siskiyou,US,CA
2,39.25666,-104.99248,Perry Park,Colorado,Douglas,US,CO
3,38.84222,-106.13113,Buena Vista,Colorado,Chaffee,US,CO
4,39.25082,-106.29252,Leadville,Colorado,Lake,US,CO


In [14]:
# dropped not required columns from DataFrame newCountyNamesDF
newCountyNamesDF = newCountyNamesDF.drop(labels=['lat','lon','name','cc'],axis=1)
newCountyNamesDF.head(5)

,State,County,state_code
0,California,Siskiyou,CA
1,California,Siskiyou,CA
2,Colorado,Douglas,CO
3,Colorado,Chaffee,CO
4,Colorado,Lake,CO


Step 7: Merge DataFrames nullCountyNameDF(with null County names) and newCountyNamesDF(with county names derived from geocoder package)

In [15]:
# merge dataframes noCountyNameDF and newCountyNamesDF

CountyNameDF = pd.merge(nullCountyNameDF, newCountyNamesDF, right_index=True,left_index=True)
CountyNameDF.head(5)

,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME,LatLong,State,County,state_code
0,8,STATE OR PRIVATE,2453437.5,67,Debris Burning,2005,0.8,B,40.968056,-122.433889,CA,NaN,"(40.96805556, -122.43388889)",California,Siskiyou,CA
1,9,STATE OR PRIVATE,2453444.5,74,Debris Burning,2005,1.0,B,41.233611,-122.283333,CA,NaN,"(41.23361111, -122.28333333)",California,Siskiyou,CA
2,48,USFS,2453535.5,165,Lightning,2005,0.5,B,39.292222,-105.183056,CO,NaN,"(39.29222222, -105.18305556)",Colorado,Douglas,CO
3,62,USFS,2453520.5,150,Lightning,2005,0.2,A,38.913333,-105.983611,CO,NaN,"(38.91333333, -105.98361111)",Colorado,Chaffee,CO
4,63,USFS,2453542.5,172,Campfire,2005,0.1,A,39.100278,-106.367500,CO,NaN,"(39.10027778, -106.3675)",Colorado,Lake,CO


In [16]:
CountyNameDF.shape

(678148, 16)

Step 8: Drop rows with the mismatching State value when DataSet value is compared with County data values obtained from reverse_geocoder package

In [17]:
CountyNameDF['Valid'] = CountyNameDF['STATE'] == CountyNameDF['state_code']
cond = CountyNameDF['Valid'] == False
CountyNameDF.drop(CountyNameDF[cond].index,inplace = True)
# reindexing the new DataFrame
CountyNameDF.index = pd.RangeIndex(len(CountyNameDF.index))

CountyNameDF.shape

(636208, 17)

In [18]:
CountyNameDF.head(5)

,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME,LatLong,State,County,state_code,Valid
0,8,STATE OR PRIVATE,2453437.5,67,Debris Burning,2005,0.8,B,40.968056,-122.433889,CA,NaN,"(40.96805556, -122.43388889)",California,Siskiyou,CA,True
1,9,STATE OR PRIVATE,2453444.5,74,Debris Burning,2005,1.0,B,41.233611,-122.283333,CA,NaN,"(41.23361111, -122.28333333)",California,Siskiyou,CA,True
2,48,USFS,2453535.5,165,Lightning,2005,0.5,B,39.292222,-105.183056,CO,NaN,"(39.29222222, -105.18305556)",Colorado,Douglas,CO,True
3,62,USFS,2453520.5,150,Lightning,2005,0.2,A,38.913333,-105.983611,CO,NaN,"(38.91333333, -105.98361111)",Colorado,Chaffee,CO,True
4,63,USFS,2453542.5,172,Campfire,2005,0.1,A,39.100278,-106.367500,CO,NaN,"(39.10027778, -106.3675)",Colorado,Lake,CO,True


In [19]:
# drop no longer required columns and rename column names as desired
CountyNameDF = CountyNameDF.drop(labels=['COUNTY_NAME','LatLong','state_code','Valid','State'],axis=1)
CountyNameDF = CountyNameDF.rename(columns={'County':'COUNTY_NAME'})
CountyNameDF.head(5)

,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME
0,8,STATE OR PRIVATE,2453437.5,67,Debris Burning,2005,0.8,B,40.968056,-122.433889,CA,Siskiyou
1,9,STATE OR PRIVATE,2453444.5,74,Debris Burning,2005,1.0,B,41.233611,-122.283333,CA,Siskiyou
2,48,USFS,2453535.5,165,Lightning,2005,0.5,B,39.292222,-105.183056,CO,Douglas
3,62,USFS,2453520.5,150,Lightning,2005,0.2,A,38.913333,-105.983611,CO,Chaffee
4,63,USFS,2453542.5,172,Campfire,2005,0.1,A,39.100278,-106.367500,CO,Lake


In [20]:
CountyNameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636208 entries, 0 to 636207
Data columns (total 12 columns):
FOD_ID              636208 non-null int64
OWNER_DESCR         636208 non-null object
DISCOVERY_DATE      636208 non-null float64
DISCOVERY_DOY       636208 non-null int64
STAT_CAUSE_DESCR    636208 non-null object
FIRE_YEAR           636208 non-null int64
FIRE_SIZE           636208 non-null float64
FIRE_SIZE_CLASS     636208 non-null object
LATITUDE            636208 non-null float64
LONGITUDE           636208 non-null float64
STATE               636208 non-null object
COUNTY_NAME         636208 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 58.2+ MB


In [21]:
US_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 12 columns):
FOD_ID              int64
OWNER_DESCR         object
DISCOVERY_DATE      float64
DISCOVERY_DOY       int64
STAT_CAUSE_DESCR    object
FIRE_YEAR           int64
FIRE_SIZE           float64
FIRE_SIZE_CLASS     object
LATITUDE            float64
LONGITUDE           float64
STATE               object
COUNTY_NAME         object
dtypes: float64(4), int64(3), object(5)
memory usage: 172.2+ MB


Step 9: removing all the rows which have no county names in DataFrame US_fire

In [22]:

# remove all the rows which have no county names in DataFrame US_fires
cond = US_fires['COUNTY_NAME'].isnull()
US_fires.drop(US_fires[cond].index,inplace = True)

# reindexing the new DataFrame
US_fires.index = pd.RangeIndex(len(US_fires.index))

In [23]:
US_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202317 entries, 0 to 1202316
Data columns (total 12 columns):
FOD_ID              1202317 non-null int64
OWNER_DESCR         1202317 non-null object
DISCOVERY_DATE      1202317 non-null float64
DISCOVERY_DOY       1202317 non-null int64
STAT_CAUSE_DESCR    1202317 non-null object
FIRE_YEAR           1202317 non-null int64
FIRE_SIZE           1202317 non-null float64
FIRE_SIZE_CLASS     1202317 non-null object
LATITUDE            1202317 non-null float64
LONGITUDE           1202317 non-null float64
STATE               1202317 non-null object
COUNTY_NAME         1202317 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 110.1+ MB


Step 10: Merge dataframes US_fires(1202317 entries) and CountyNameDF(636208 entries)

In [24]:
US_fires = pd.concat([US_fires, CountyNameDF])

In [25]:
US_fires


,FOD_ID,OWNER_DESCR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME
0,1,USFS,2453403.5,33,Miscellaneous,2005,0.10,A,40.036944,-121.005833,CA,Plumas
1,2,USFS,2453137.5,133,Lightning,2004,0.25,A,38.933056,-120.404444,CA,Placer
2,3,STATE OR PRIVATE,2453156.5,152,Debris Burning,2004,0.10,A,38.984167,-120.735556,CA,El Dorado
3,4,USFS,2453184.5,180,Lightning,2004,0.10,A,38.559167,-119.913333,CA,Alpine
4,5,USFS,2453184.5,180,Lightning,2004,0.10,A,38.559167,-119.933056,CA,Alpine
...,...,...,...,...,...,...,...,...,...,...,...,...
636203,300348363,STATE OR PRIVATE,2457291.5,269,Missing/Undefined,2015,0.01,A,40.481637,-122.389375,CA,Shasta
636204,300348373,MUNICIPAL/LOCAL,2457300.5,278,Miscellaneous,2015,0.20,A,37.617619,-120.938570,CA,Stanislaus
636205,300348375,MUNICIPAL/LOCAL,2457144.5,122,Missing/Undefined,2015,0.10,A,37.617619,-120.938570,CA,Stanislaus
636206,300348377,MUNICIPAL/LOCAL,2457309.5,287,Missing/Undefined,2015,2.00,B,37.672235,-120.898356,CA,Stanislaus


In [26]:
US_fires.shape

(1838525, 12)

Step 11: column "DISCOVERY_DATE" was in data type float64 and not in Datetime format as expected. Hence I converted it to a readable date format from julian date.

In [27]:
from datetime import datetime
epoch = pd.to_datetime(0, unit='s').to_julian_date()
US_fires['date'] = pd.to_datetime(US_fires['DISCOVERY_DATE']- epoch, unit='D')

In [28]:
US_fires[['date','DISCOVERY_DATE']]

,date,DISCOVERY_DATE
0,2005-02-02,2453403.5
1,2004-05-12,2453137.5
2,2004-05-31,2453156.5
3,2004-06-28,2453184.5
4,2004-06-28,2453184.5
...,...,...
636203,2015-09-26,2457291.5
636204,2015-10-05,2457300.5
636205,2015-05-02,2457144.5
636206,2015-10-14,2457309.5


Step 12: Dropped DISCOVERY_DATE. Remaned "date" column and FOD_ID column names to more appropriate names.        

In [29]:
US_fires = US_fires.drop(labels=['DISCOVERY_DATE'],axis=1)
US_fires = US_fires.rename(columns={'date':'DISCOVERY_DATE','FOD_ID':'GLOBAL_UNIQUE_ID'})
US_fires.head()

,GLOBAL_UNIQUE_ID,OWNER_DESCR,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME,DISCOVERY_DATE
0,1,USFS,33,Miscellaneous,2005,0.10,A,40.036944,-121.005833,CA,Plumas,2005-02-02
1,2,USFS,133,Lightning,2004,0.25,A,38.933056,-120.404444,CA,Placer,2004-05-12
2,3,STATE OR PRIVATE,152,Debris Burning,2004,0.10,A,38.984167,-120.735556,CA,El Dorado,2004-05-31
3,4,USFS,180,Lightning,2004,0.10,A,38.559167,-119.913333,CA,Alpine,2004-06-28
4,5,USFS,180,Lightning,2004,0.10,A,38.559167,-119.933056,CA,Alpine,2004-06-28


Step 13: Created a column with full state Name as value

In [30]:
# Build a dictionary with State names as value and its Abbreviation as key.
statenameDict = dict(zip( zipcode['Abbreviation'],zipcode['State']))
    
    
# map State names from dataFrame newCountyNamesDF with dictionary zipcodeDict
US_fires['STATE_NAME'] = US_fires['STATE'].map(statenameDict)
US_fires.head(5)

,GLOBAL_UNIQUE_ID,OWNER_DESCR,DISCOVERY_DOY,STAT_CAUSE_DESCR,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY_NAME,DISCOVERY_DATE,STATE_NAME
0,1,USFS,33,Miscellaneous,2005,0.10,A,40.036944,-121.005833,CA,Plumas,2005-02-02,California
1,2,USFS,133,Lightning,2004,0.25,A,38.933056,-120.404444,CA,Placer,2004-05-12,California
2,3,STATE OR PRIVATE,152,Debris Burning,2004,0.10,A,38.984167,-120.735556,CA,El Dorado,2004-05-31,California
3,4,USFS,180,Lightning,2004,0.10,A,38.559167,-119.913333,CA,Alpine,2004-06-28,California
4,5,USFS,180,Lightning,2004,0.10,A,38.559167,-119.933056,CA,Alpine,2004-06-28,California


Step 14: Rearraged columns in dataframe

In [31]:
column_names = ["GLOBAL_UNIQUE_ID", "DISCOVERY_DATE",'FIRE_YEAR','STAT_CAUSE_DESCR','FIRE_SIZE','FIRE_SIZE_CLASS','STATE','STATE_NAME','COUNTY_NAME','LATITUDE','LONGITUDE','DISCOVERY_DOY','OWNER_DESCR']
US_fires = US_fires.reindex(columns=column_names)

In [32]:
US_fires.head()

,GLOBAL_UNIQUE_ID,DISCOVERY_DATE,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE,STATE_NAME,COUNTY_NAME,LATITUDE,LONGITUDE,DISCOVERY_DOY,OWNER_DESCR
0,1,2005-02-02,2005,Miscellaneous,0.10,A,CA,California,Plumas,40.036944,-121.005833,33,USFS
1,2,2004-05-12,2004,Lightning,0.25,A,CA,California,Placer,38.933056,-120.404444,133,USFS
2,3,2004-05-31,2004,Debris Burning,0.10,A,CA,California,El Dorado,38.984167,-120.735556,152,STATE OR PRIVATE
3,4,2004-06-28,2004,Lightning,0.10,A,CA,California,Alpine,38.559167,-119.913333,180,USFS
4,5,2004-06-28,2004,Lightning,0.10,A,CA,California,Alpine,38.559167,-119.933056,180,USFS


In [33]:
US_fires.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1838525 entries, 0 to 636207
Data columns (total 13 columns):
GLOBAL_UNIQUE_ID    int64
DISCOVERY_DATE      datetime64[ns]
FIRE_YEAR           int64
STAT_CAUSE_DESCR    object
FIRE_SIZE           float64
FIRE_SIZE_CLASS     object
STATE               object
STATE_NAME          object
COUNTY_NAME         object
LATITUDE            float64
LONGITUDE           float64
DISCOVERY_DOY       int64
OWNER_DESCR         object
dtypes: datetime64[ns](1), float64(3), int64(3), object(6)
memory usage: 196.4+ MB


Step 15: Checked if there are any outliers with respect to FIRE_YEAR column

In [34]:
US_fires[['FIRE_YEAR']].describe()

,FIRE_YEAR
count,1.838525e+06
mean,2.003717e+03
std,6.688871e+00
min,1.992000e+03
25%,1.998000e+03
50%,2.004000e+03
75%,2.009000e+03
max,2.015000e+03


there were no outliers which needs to be discarded

In [35]:
US_fires['COUNTY_NAME'].value_counts(dropna=False)

Riverside          19398
Lincoln            17928
Jackson            14460
Coconino           13864
Washington         13152
                   ...  
Concordia              1
Richmond (city)        1
Terrebonne             1
Harmon                 1
                       1
Name: COUNTY_NAME, Length: 1882, dtype: int64

Created a column FIPS_CODE with FIPS code value for each County Name.

In [36]:
US_FIPS_CODE = pd.read_csv("County_FIPS_Code_Data.csv")
US_FIPS_CODE["FIPS"] = US_FIPS_CODE.FIPS.map("{:05}".format)
US_FIPS_CODE['Code'] = list(zip(US_FIPS_CODE.State, US_FIPS_CODE.Name))
US_FIPS_CODE.head()

,FIPS,Name,State,Code
0,01001,Autauga,AL,"(AL, Autauga)"
1,01003,Baldwin,AL,"(AL, Baldwin)"
2,01005,Barbour,AL,"(AL, Barbour)"
3,01007,Bibb,AL,"(AL, Bibb)"
4,01009,Blount,AL,"(AL, Blount)"


In [37]:
zipcodeDict = dict(zip(US_FIPS_CODE['Code'], US_FIPS_CODE['FIPS']))

In [39]:
US_fires['Code'] = list(zip(US_fires.STATE, US_fires.COUNTY_NAME))
US_fires['FIPS_CODE'] = US_fires['Code'].map(zipcodeDict)
US_fires.head(5)

,GLOBAL_UNIQUE_ID,DISCOVERY_DATE,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE,STATE_NAME,COUNTY_NAME,LATITUDE,LONGITUDE,DISCOVERY_DOY,OWNER_DESCR,Code,FIPS_CODE
0,1,2005-02-02,2005,Miscellaneous,0.10,A,CA,California,Plumas,40.036944,-121.005833,33,USFS,"(CA, Plumas)",06063
1,2,2004-05-12,2004,Lightning,0.25,A,CA,California,Placer,38.933056,-120.404444,133,USFS,"(CA, Placer)",06061
2,3,2004-05-31,2004,Debris Burning,0.10,A,CA,California,El Dorado,38.984167,-120.735556,152,STATE OR PRIVATE,"(CA, El Dorado)",06017
3,4,2004-06-28,2004,Lightning,0.10,A,CA,California,Alpine,38.559167,-119.913333,180,USFS,"(CA, Alpine)",06003
4,5,2004-06-28,2004,Lightning,0.10,A,CA,California,Alpine,38.559167,-119.933056,180,USFS,"(CA, Alpine)",06003


In [40]:
US_fires = US_fires.drop(labels=['Code'],axis=1)

In [41]:
US_fires.to_csv("US_Widfires_cleanData.csv")